In [58]:
import pytorch

ModuleNotFoundError: No module named 'pytorch'

In [56]:
# testing lib fn
from __future__ import print_function, unicode_literals
from unicodedata import normalize
import library_py2 as lib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.collocations import *
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, Masking
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import argparse
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [28]:
df = pd.read_pickle('preprocessed_data2.pkl')

In [29]:
df.head()

,context,question,answers,plausible_answers,is_impossible,context_lemma_pos,question_lemma_pos,answers_lemma_pos,plausible_answers_lemma_pos,context_lemma_pos_tfidf,question_lemma_pos_tfidf
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(when, WRB, 0.2), (beyonce, NN, 0.2), (start,..."
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(what, WP, 0.2), (area, NNS, 0.2), (beyonce, ..."
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(when, WRB, 0.125), (beyonce, NN, 0.125), (le..."
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(in, IN, 0.2), (city, NN, 0.2), (state, NN, 0..."
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(in, IN, 0.2), (decade, NN, 0.2), (beyonce, N..."


In [30]:
len(df)

130319

In [31]:
df = df.loc[df.is_impossible==0]

In [32]:
len(df)

86821

In [33]:
df['answer_len']=df.answers_lemma_pos.apply(lambda x: len(x))
df['question_len']=df.question_lemma_pos.apply(lambda x: len(x))
df['context_len']=df.context_lemma_pos.apply(lambda x: len(x))


/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
df = df.drop(columns=['context_lemma_pos_tfidf', 'question_lemma_pos_tfidf', 'plausible_answers', 'plausible_answers_lemma_pos', 'answers', 'question','context'])

In [47]:
df.head()

,is_impossible,context_lemma_pos,question_lemma_pos,answers_lemma_pos,answer_len,question_len,context_len
0,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",2,5,75
1,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",2,5,75
2,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",1,8,75
3,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",2,5,75
4,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",2,5,75


In [48]:
df.drop(columns=['is_impossible'], inplace=True)

In [53]:
df1_pos_W = df.loc[df.answer_len==1]

In [55]:
df1_pos_W.question_lemma_pos

2         [(when, WRB), (beyonce, NN), (leave, VBP), (de...
11        [(when, WRB), (beyoncé, NN), (release, NN), (d...
12        [(how, WRB), (many, JJ), (grammy, JJ), (award,...
15        [(after, IN), (second, JJ), (solo, NN), (album...
17        [(to, TO), (set, VB), (record, NN), (grammys, ...
                                ...                        
130044    [(from, IN), (city, NN), (arkefly, NN), (offer...
130046    [(in, IN), (us, PRP), (state, NN), (kathmandu,...
130047    [(what, WP), (yangon, VBZ), (previously, RB), ...
130048    [(with, IN), (belorussian, JJ), (city, NN), (k...
130049    [(in, IN), (year, NN), (kathmandu, NN), (creat...
Name: question_lemma_pos, Length: 31989, dtype: object

In [62]:
pos_dict=lib.get_pos_dict(df1_pos_W,columns=['context_lemma_pos'])
pos_dict

{'': 0.0,
 'NN': 0.9592715737637746,
 'VBP': 0.360315972751275,
 'JJ': 0.927388867444324,
 'CD': 0.7224863933266896,
 'NNS': 0.8598924378549789,
 'VBD': 0.9558813218777534,
 'VBG': 0.4588106026733826,
 'RB': 0.8611914183331262,
 'VBZ': 0.6961802733464587,
 'VBN': 0.6132534909840617,
 'PRP$': 0.4684079411393567,
 'VB': 0.07087393936983433,
 'DT': 0.1976010461289125,
 'PRP': 0.48297460721994667,
 'JJR': 0.9573565640048951,
 'IN': 0.9205742559261008,
 'JJS': 0.022584915446127063,
 'NNP': 0.7180885585022406,
 'MD': 0.12392201019381988,
 'WDT': 0.26916270907838447,
 'FW': 0.35981720808378537,
 '$': 0.9628612911409422,
 'WRB': 0.6699031132055633,
 'RP': 0.22522257785126742,
 'RBR': 0.3444770078816314,
 'TO': 0.8305390934361616,
 'RBS': 0.20702044303076117,
 'CC': 0.16964627591060644,
 'WP$': 0.13086722306185827,
 'WP': 0.7444226774856989,
 'EX': 0.38431416596916834,
 'NNPS': 0.8220249472354502,
 'PDT': 0.1729198618724923,
 'SYM': 0.7412727310019118,
 'POS': 0.6552685606251134,
 'UH': 0.29439

In [63]:
len(df1_pos_W)

31989

In [64]:
df1_pos_W.to_pickle('df1_pos_W.pkl')